# Simple CIPHER Design Notebook

This notebook provides a simple way to run CIPHER without needing shell scripts. It includes:
1. Data formatting
2. Parameter setup
3. Running CIPHER to design probes

**Note**: This notebook assumes you are running on a compute node with the conda environment activated.

## Step 1: Setup and Imports

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import anndata

# Add Design directory to path
design_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.getcwd()
sys.path.insert(0, design_dir)
from CIPHER import CIPHER

## Step 2: Configure Paths and Data

**Update these paths to match your data location:**

In [ ]:
# ===== CONFIGURE THESE PATHS =====
# Path to your gene expression data (h5ad file or similar)
data_path = '/path/to/your/data.h5ad'

# Path to probe constraints file (CSV with gene names and max probes per gene)
# Format: CSV with 'gene' column and 'probes' or similar column
constraints_file = '/path/to/probe_constraints.csv'

# Output directory for formatted data and results
output_dir = './cipher_output'
os.makedirs(output_dir, exist_ok=True)

# Cell type label column name in your data
cell_type_label = 'cell_type'  # Update to match your metadata column

# ===== END CONFIGURATION =====

print(f"Data path: {data_path}")
print(f"Output directory: {output_dir}")

## Step 3: Load and Format Data

This section loads your data and formats it into the required format for CIPHER.

In [ ]:
# Load expression data
print("Loading expression data...")
adata = anndata.read_h5ad(data_path)
print(f"Loaded data: {adata.shape[0]} cells, {adata.shape[1]} genes")

# Get gene expression matrix
if isinstance(adata.X, np.ndarray):
    X = torch.tensor(adata.X, dtype=torch.float32)
else:
    # If sparse matrix, convert to dense
    X = torch.tensor(adata.X.todense(), dtype=torch.float32)

# Normalize by library size (optional, adjust as needed)
if 'library_size' in adata.obs.columns:
    correction = torch.tensor(100000 / np.array(adata.obs['library_size']))
    X = X * correction[:, None]

# Get cell type labels
if cell_type_label not in adata.obs.columns:
    raise ValueError(f"Cell type label '{cell_type_label}' not found in data. Available columns: {adata.obs.columns.tolist()}")

cell_types = adata.obs[cell_type_label].unique()
categorical_converter = {k: i for i, k in enumerate(sorted(cell_types))}
y = torch.tensor(np.array([categorical_converter[ct] for ct in adata.obs[cell_type_label]]), dtype=torch.long)

print(f"Found {len(cell_types)} cell types: {sorted(cell_types)}")
print(f"Expression matrix shape: {X.shape}")

In [ ]:
# Load or create gene constraints
print("Loading gene constraints...")
if os.path.exists(constraints_file):
    constraints_df = pd.read_csv(constraints_file)
    # Try to find gene column and probe count column
    gene_col = None
    probe_col = None
    for col in constraints_df.columns:
        if 'gene' in col.lower():
            gene_col = col
        if 'probe' in col.lower() or 'constraint' in col.lower():
            probe_col = col
    
    if gene_col is None or probe_col is None:
        # Assume first column is gene, second is constraint
        gene_col = constraints_df.columns[0]
        probe_col = constraints_df.columns[1]
    
    # Create mapping from gene names to constraints
    gene_to_constraint = dict(zip(constraints_df[gene_col], constraints_df[probe_col]))
    
    # Match constraints to genes in expression data
    gene_names = adata.var_names.tolist()
    constraints = np.array([gene_to_constraint.get(gene, 0) for gene in gene_names])
    constraints = np.clip(constraints, 0, 100)  # Cap at 100 probes per gene
else:
    print(f"Constraints file not found at {constraints_file}")
    print("Creating default constraints (100 probes per gene)...")
    constraints = np.ones(X.shape[1]) * 100

print(f"Constraints: min={constraints.min()}, max={constraints.max()}, mean={constraints.mean():.1f}")

In [ ]:
# Split into train and test sets
print("Splitting data into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} cells")
print(f"Test set: {X_test.shape[0]} cells")

# Save formatted data
torch.save(X_train, os.path.join(output_dir, 'X_train.pt'))
torch.save(X_test, os.path.join(output_dir, 'X_test.pt'))
torch.save(y_train, os.path.join(output_dir, 'y_train.pt'))
torch.save(y_test, os.path.join(output_dir, 'y_test.pt'))

# Save constraints
constraints_df = pd.DataFrame(constraints, index=adata.var_names, columns=['constraints'])
constraints_df.to_csv(os.path.join(output_dir, 'constraints.csv'))

# Save label converter
label_converter_df = pd.DataFrame({
    'index': list(categorical_converter.values()),
    'cell_type': list(categorical_converter.keys())
})
label_converter_df.to_csv(os.path.join(output_dir, 'categorical_converter.csv'), index=False)

print(f"\nSaved formatted data to {output_dir}")
print("Files created:")
print("  - X_train.pt, X_test.pt")
print("  - y_train.pt, y_test.pt")
print("  - constraints.csv")
print("  - categorical_converter.csv")

## Step 4: Set Up Parameters

Configure CIPHER parameters. Adjust these based on your needs.

In [ ]:
# Create parameters dictionary
parameters = {
    # Core model parameters
    'n_cpu': 3,
    'n_bit': 24,  # Number of bits in encoding
    'n_iters': 10000,  # Training iterations (increase for better results)
    'batch_size': 500,  # Batch size (0 = use full dataset)
    
    # Loss function weights
    'categorical_wt': 2.5,  # Classification accuracy weight
    'probe_wt': 1,  # Probe count constraint weight
    'gene_constraint_wt': 1,  # Gene constraint violation weight
    'brightness_wt': 1,  # Target brightness weight
    'dynamic_wt': 1,  # Dynamic range weight
    'separation_wt': 1,  # Cell type separation weight
    
    # Target parameters (with _s/_e for dynamic targets)
    'brightness_s': 4.5,  # Initial target brightness (log10 scale)
    'brightness_e': 4.5,  # Final target brightness
    'dynamic_fold_s': 2.0,  # Initial dynamic range fold-change
    'dynamic_fold_e': 2.0,  # Final dynamic range fold-change
    'separation_fold_s': 1.0,  # Initial separation fold-change
    'separation_fold_e': 1.0,  # Final separation fold-change
    'n_probes': 50000,  # Target total number of probes
    
    # Training parameters
    'lr_s': 0.01,  # Initial learning rate
    'lr_e': 0.01,  # Final learning rate
    'saturation': 1.0,  # When to reach final _e values (0.0-1.0)
    'gradient_clip': 1,  # Gradient clipping
    'report_rt': 500,  # Report progress every N iterations
    
    # Model architecture
    'decoder_n_lyr': 1,  # Number of decoder hidden layers
    'encoder_act': 'tanh',  # Encoder activation
    'decoder_act': 'gelu',  # Decoder activation
    
    # Data paths (relative to output_dir)
    'input': output_dir,
    'X_train': os.path.join(output_dir, 'X_train.pt'),
    'X_test': os.path.join(output_dir, 'X_test.pt'),
    'y_train': os.path.join(output_dir, 'y_train.pt'),
    'y_test': os.path.join(output_dir, 'y_test.pt'),
    'constraints': os.path.join(output_dir, 'constraints.csv'),
    'y_label_converter_path': os.path.join(output_dir, 'categorical_converter.csv'),
    
    # Output directory
    'output': os.path.join(output_dir, 'design_results'),
    
    # Other parameters
    'device': 'cpu',
    'top_n_genes': 0,  # 0 = use all genes
    'sum_norm': 0,  # Sum normalization
    'bit_norm': 0,  # Bit-wise normalization
    'use_noise': 1,  # Enable noise during training
    'continue_training': 0,  # Don't continue if model loaded
    'best_model': 1,  # Save best model
    'sparsity_s': 0.95,  # Target sparsity
    'sparsity_e': 0.95,
    'sparsity_wt': 0,  # Sparsity weight (0 = disabled)
    'label_smoothing': 0.1,  # Label smoothing for classification
    'gene_importance_wt': 0,  # Gene importance weight
    'gene_importance': 0.25,  # Max gene contribution per bit
    'bit_usage_wt': 0,  # Bit usage weight
    'bit_usage': 0.1,
    'bit_corr_wt': 0,  # Bit correlation weight
    'bit_corr': 0.8,
    'step_size_wt': 0,  # Step size weight
    'step_size_threshold': 1e2,
    'step_size_n_steps': 0.1,
    'P_scaling': 24,  # Projection scaling
    
    # Noise parameters (all set to 0 for simplicity, can enable for robustness)
    'X_drp_s': 0, 'X_drp_e': 0,
    'X_noise_s': 0, 'X_noise_e': 0,
    'E_drp_s': 0, 'E_drp_e': 0,
    'E_noise_s': 0, 'E_noise_e': 0,
    'P_drp_s': 0, 'P_drp_e': 0,
    'P_noise_s': 0, 'P_noise_e': 0,
    'P_add_s': 4.0, 'P_add_e': 4.0,  # Background signal
    'D_drp_s': 0, 'D_drp_e': 0,
    'E_perturb_rt': 0,  # Weight perturbation
    'E_perb_prct': 0.01,
    'E_init_min': 0.001, 'E_init_max': 0.01,
    'E_perturb_min': 0.05, 'E_perturb_max': 0.25,
    'central_brain': 0,
}

# Save parameters to CSV file
param_file = os.path.join(output_dir, 'parameters.csv')
param_df = pd.DataFrame({'values': list(parameters.values())}, index=pd.Index(list(parameters.keys())))
param_df.to_csv(param_file)

print(f"Parameters saved to {param_file}")
print(f"\nKey parameters:")
print(f"  - n_bit: {parameters['n_bit']}")
print(f"  - n_iters: {parameters['n_iters']}")
print(f"  - n_probes: {parameters['n_probes']}")
print(f"  - decoder_n_lyr: {parameters['decoder_n_lyr']}")

## Step 5: Run CIPHER

Initialize and train the CIPHER model.

In [ ]:
# Initialize CIPHER model
print("Initializing CIPHER model...")
model = CIPHER(user_parameters_path=param_file)

# Initialize the model (loads data, sets up encoder/decoder)
if not model.initialize():
    print("Initialization failed. Check the log file for details.")
    raise RuntimeError("Model initialization failed")

print("Model initialized successfully!")
print(f"  - Number of genes: {model.n_genes}")
print(f"  - Number of bits: {model.I['n_bit']}")
print(f"  - Number of cell types: {model.n_categories}")

In [ ]:
# Train the model
print("\nStarting training...")
print(f"This will run for {parameters['n_iters']} iterations.")
print(f"Progress will be reported every {parameters['report_rt']} iterations.")
print(f"Results will be saved to: {model.I['output']}")

model.fit()

print("\nTraining completed!")

In [ ]:
# Run evaluation
print("Running evaluation...")
model.evaluate()

print(f"\nEvaluation complete!")
print(f"Results saved to: {model.I['output']}")
print("\nKey output files:")
print("  - learning_stats.csv: Training statistics")
print("  - evaluation_results.csv: Final evaluation metrics")
print("  - E_weights.csv: Final encoding weights (probe design)")
print("  - model_best.pt: Best model checkpoint")
print("  - comprehensive_performance.pdf: Training curves")